In [47]:
import pandas as pd
import numpy as np
# а также импортируем модули Seaborn и Matplotlib для работы с графикой.
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML
import datetime as datetime
# %load_ext autoreload
# %autoreload 2


from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import sys
import os
from graphviz import Source
from IPython.display import SVG

%matplotlib inline

from tqdm import tqdm, tqdm_notebook
import datetime

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix


sys.path.append(os.path.join(sys.path[0], '../'))

from sklearn.ensemble import AdaBoostClassifier

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
style = "<style>svg{width:30% !important;height:30% !important;}</style>"
HTML(style)

# import hierarchical clustering libraries
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering

In [48]:
# %%telegram_send "Эпохи обработаны"!
import telepot
%load_ext ipytelegram
%reload_ext ipytelegram
%telegram_setup <AAFyFIYwdtDL_w1EK9t5X89H66vK_N-Fys> <1859374621>
# bot = telepot.bot('1859374621:AAFyFIYwdtDL_w1EK9t5X89H66vK_N-FysU')
# bot.getUpdates()['mesage']['chat']
%telegram_setup 1859374621:AAFyFIYwdtDL_w1EK9t5X89H66vK_N-FysU 1338353058

The ipytelegram extension is already loaded. To reload it, use:
  %reload_ext ipytelegram


In [49]:
xls = pd.read_excel('df.xlsx')

# Готовим данные

In [50]:
df_pp = xls.copy(deep=True)
df_pp = df_pp.loc[:, ~df_pp.columns.str.contains('^Unnamed')]
df_pp = df_pp.replace(to_replace =',', value = '.', regex = True)
#df.rename(columns={'Потребкредит': 'type_credit'}, inplace=True)
df_pp = df_pp.fillna(0)
#df_pp.dtypes
#df_pp.columns

In [53]:
# df = df_pp.drop(['№', 'name', 
#        'legal', 'soft', 'sale', 'arhive'], axis = 1)

# #df.dtypes
# df.columns

In [55]:
# df_ds = pd.get_dummies(df, columns=["type"])
# #df_ds = pd.get_dummies(df_ds, columns=["region"])
# # 
# #df_ds.head()




In [56]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 

In [57]:
df_train = df_ds[df_ds['case'].isin(["Сбер3"])] #


# Отбросим колонку, которую будем предсказывать.
X_train = df_train.drop([ 'income_sum', 'income', 'target', 'tg_grade', 'case'], axis=1)
# Создадим переменную, которую будем предсказывать.
y_train = df_train.target


df_test = df_ds[df_ds['case'].isin(["Сбер4"])] #


# Отбросим колонку, которую будем предсказывать.
X_test = df_test.drop([ 'income_sum', 'income', 'target', 'tg_grade', 'case'], axis=1)
# Создадим переменную, которую будем предсказывать.
y_test = df_test.target


df_ros = df_ds[df_ds['case'].isin(["Росбанк1"])] #


# Отбросим колонку, которую будем предсказывать.
X_ros = df_ros.drop([ 'income_sum', 'income', 'target', 'tg_grade', 'case'], axis=1)
# Создадим переменную, которую будем предсказывать.
y_ros = df_ros.target
#X_t

# x_train.fillna(0)
# x_test.fillna(0)

x_train = X_train
y_train = y_train
x_test = X_test
y_test = y_test

In [58]:
# x_train
# # y_train
# x_test
# # y_test

In [59]:
import pandas as pd
import numpy as np
import os
import tqdm
import pickle


from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score

from catboost import CatBoostClassifier, Pool
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

np.warnings.filterwarnings('ignore')

In [60]:
cv = KFold(n_splits=5, random_state=100, shuffle=True)
#%%time

oof = np.zeros(len(x_train))
train_preds = np.zeros(len(x_train))

models = []

tree_params = {
    'max_depth': 5,
    'eval_metric': 'AUC',
    'loss_function': 'Logloss',
    'random_state': 100,
    'l2_leaf_reg': 1,
    #'task_type': 'GPU' # если имеется GPU с CUDA, то можно ее задействовать, чтобы на порядок ускорить обучение
}


for fold_, (train_idx, val_idx) in enumerate(cv.split(x_train, y_train), 1):
    print(f'Началось обучение на фолде номер:= {fold_}.')
    model = CatBoostClassifier(**tree_params)    
    
    train, val = x_train.iloc[train_idx], x_train.iloc[val_idx]
    
    train_pool = Pool(x_train, y_train.values)
    val_pool = Pool(x_train, y_train.values)
    

    model.fit(train_pool, eval_set=[val_pool], early_stopping_rounds=100, verbose_eval=50, use_best_model=True, plot=False)
    
    oof[val_idx] = model.predict_proba(val_pool)[:, 1]
    
    train_preds[train_idx] += model.predict_proba(train_pool)[:, 1] / (cv.n_splits-1)
    models.append(model)
    
    file_name_model = f'model_CatBoost/model_{name_model}_{fold_}'
    model.save_model(PATH_TO_WORKDIR + file_name_model)
    print(f'Обучение на фолде номер:= {fold_} завершилось.')


Началось обучение на фолде номер:= 1.
0:	test: 0.6875450	best: 0.6875450 (0)	total: 3.52ms	remaining: 3.52s
50:	test: 0.7243082	best: 0.7243082 (50)	total: 160ms	remaining: 2.98s
100:	test: 0.7492494	best: 0.7492494 (100)	total: 328ms	remaining: 2.92s
150:	test: 0.7652345	best: 0.7652345 (150)	total: 494ms	remaining: 2.77s
200:	test: 0.7819454	best: 0.7819454 (200)	total: 670ms	remaining: 2.66s
250:	test: 0.7959004	best: 0.7959004 (250)	total: 847ms	remaining: 2.53s
300:	test: 0.8105308	best: 0.8105308 (300)	total: 1.03s	remaining: 2.4s
350:	test: 0.8252362	best: 0.8252362 (350)	total: 1.23s	remaining: 2.27s
400:	test: 0.8395277	best: 0.8395277 (400)	total: 1.47s	remaining: 2.2s
450:	test: 0.8551739	best: 0.8551739 (450)	total: 1.78s	remaining: 2.17s
500:	test: 0.8677216	best: 0.8677216 (500)	total: 2.17s	remaining: 2.17s
550:	test: 0.8794759	best: 0.8794759 (550)	total: 2.41s	remaining: 1.96s
600:	test: 0.8907638	best: 0.8907638 (600)	total: 2.63s	remaining: 1.75s
650:	test: 0.8996355

ValueError: shape mismatch: value array of shape (3075,) could not be broadcast to indexing result of shape (615,)

In [61]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # sigmoid, т.к. данные от 0 до 1

In [62]:
print(model.summary()) # архитектура нашей модели

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                1728      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 5,953
Trainable params: 5,953
Non-trainable params: 0
_________________________________________________________________
None


In [63]:



pred = model.predict(X_ros).flatten() 


#predictions = scaler.inverse_transform(predictions)
X_ros['pred'] = pred


writer = pd.ExcelWriter('X_ros_V0.1.xlsx')
X_ros.to_excel(writer)
writer.save()